In [ ]:
!pip install rdkit
!pip install shap
!pip install Catboost
!pip install torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# import packages and modules

from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, Descriptors, AllChem
from rdkit.Chem.Draw import IPythonConsole
from itertools import combinations
import IPython
from IPython.display import display, Image as IPImage
from PIL import Image
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import re
from rdkit import Chem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, cohen_kappa_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import rdmolops
from scipy.stats import pearsonr
from rdkit.Chem import rdchem
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader, Batch
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool


In [ ]:
# Read Input files

train_df = pd.read_excel('/content/drive/MyDrive/HOB/input_gnn_20.xlsx', sheet_name = "train")
print(train_df.shape)

test_df = pd.read_excel('/content/drive/MyDrive/HOB/input_gnn_20.xlsx', sheet_name = "test")
print(test_df.shape)

(697, 43)
(45, 43)


In [ ]:
# ATC code frequency matrix
atc_train = pd.read_excel('/content/drive/MyDrive/HOB/drug_atc.xlsx', sheet_name='train_final')
atc_test = pd.read_excel('/content/drive/MyDrive/HOB/drug_atc.xlsx', sheet_name='test')
atc_train_filled = atc_train[['Drug Name', 'ATC_Codes']].fillna('0').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
atc_test_filled = atc_test[['Drug Name', 'ATC_Codes']].fillna('0').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
atc_codes = pd.concat([atc_train_filled, atc_test_filled], axis=0).reset_index(drop=True)

# Function to split ATC codes
def split_atc_codes(atc_codes):
    if atc_codes == '0':
        return ['0']
    atc_codes = atc_codes.strip()
    parts = []
    for code in atc_codes.split(', '):
        code = code.strip()
        if len(code) >= 1:
            parts.append(code[0])  # Single char
        if len(code) >= 3:
            parts.append(code[:3])  # Single char and two digits
        if len(code) >= 4:
            parts.append(code[:4])  # Single char, two digits, single char
        if len(code) >= 5:
            parts.append(code[:5])  # Single char, two digits, two chars
        parts.append(code)  # Entire string
    return parts

# Apply split function and get unique ATC code parts
all_atc_parts = atc_codes['ATC_Codes'].apply(split_atc_codes)
unique_atc_parts = set(part for sublist in all_atc_parts for part in sublist)
unique_atc_parts = sorted(unique_atc_parts)

unique_atc_parts = [code for code in unique_atc_parts if code != '0']

atc_matrix_train = pd.DataFrame(0, index = atc_train['Drug Name'], columns = unique_atc_parts)
atc_matrix_test = pd.DataFrame(0, index = atc_test['Drug Name'], columns = unique_atc_parts)

In [ ]:
atc_matrix_train = pd.DataFrame(0, index=atc_train['Drug Name'], columns=unique_atc_parts)

# Assuming atc_train_filled is a DataFrame or converting if it's not
if not isinstance(atc_train_filled, pd.DataFrame):
    atc_train_filled = pd.DataFrame(atc_train_filled)

# Now iterate over rows in the DataFrame
for i, row in atc_train_filled.iterrows():
    atc_codes = split_atc_codes(row['ATC_Codes'])
   # print(atc_codes)
    for code in atc_codes:
        if code in unique_atc_parts:
            atc_matrix_train.at[row['Drug Name'], code] += 1

# Fill NaN or missing values with 0
atc_matrix_train.fillna(0, inplace=True)

# Display the resulting matrix
#print(atc_matrix_train)
print(atc_matrix_train.shape)

(698, 2088)


In [ ]:
# Index of the row to delete
index_to_delete = 'Selenium'

# Drop the row with the specified index
atc_matrix_train = atc_matrix_train.drop(index_to_delete)

# Reset index if needed
#drug_embeddings_df.reset_index(drop=True, inplace=True)


In [ ]:
atc_matrix_test = pd.DataFrame(0, index=atc_test['Drug Name'], columns=unique_atc_parts)

# Assuming atc_test_filled is a DataFrame or converting if it's not
if not isinstance(atc_test_filled, pd.DataFrame):
    atc_test_filled = pd.DataFrame(atc_test_filled)

# Now iterate over rows in the DataFrame
for i, row in atc_test_filled.iterrows():
    atc_codes = split_atc_codes(row['ATC_Codes'])
    #print(atc_codes)
    for code in atc_codes:
        if code in unique_atc_parts:
            atc_matrix_test.at[row['Drug Name'], code] += 1

# Fill NaN or missing values with 0
atc_matrix_test.fillna(0, inplace=True)

# Display the resulting matrix
#print(atc_matrix_test)
print(atc_matrix_test.shape)

(45, 2088)


In [ ]:
def preprocess_classification_atc(X):
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    scaler = MinMaxScaler()
    #scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    return X_scaled

train_atc_norm = preprocess_classification_atc(atc_matrix_train.iloc[:, 0:])
test_atc_norm = preprocess_classification_atc(atc_matrix_test.iloc[:, 0:])

In [ ]:
cols = [f'atc_{i}' for i in range(atc_matrix_train.shape[1])]
train_atc_norm_df = pd.DataFrame(train_atc_norm, columns=cols)
#print(train_atc_norm_df.head())


cols = [f'atc_{i}' for i in range(atc_matrix_test.shape[1])]
test_atc_norm_df = pd.DataFrame(test_atc_norm, columns=cols)
#print(test_atc_norm_df.head())

In [ ]:
'''
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdchem
import matplotlib.pyplot as plt
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Batch

# Node feature encoding for hybridization
def encode_hybridization(hybridization):
    hybridization_map = {
        rdchem.HybridizationType.SP: 0,
        rdchem.HybridizationType.SP2: 1,
        rdchem.HybridizationType.SP3: 2,
        rdchem.HybridizationType.SP3D: 3,
        rdchem.HybridizationType.SP3D2: 4,
        rdchem.HybridizationType.UNSPECIFIED: 5,
    }
    return hybridization_map.get(hybridization, 5)

# Convert SMILES to graph data object
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES: {smiles}")
        return None

    # Handle single atom or molecule with no bonds
    if mol.GetNumBonds() == 0:
        print(f"SMILES has no bonds: {smiles}")
        return None

    # Node features
    node_features = []
    for atom in mol.GetAtoms():
        feature = [
            atom.GetAtomicNum(),
            atom.GetDegree(),
            atom.GetTotalNumHs(),
            atom.GetImplicitValence(),
            int(atom.GetIsAromatic()),
            encode_hybridization(atom.GetHybridization())
        ]
        node_features.append(feature)
    node_features = torch.tensor(node_features, dtype=torch.float)

    # Edge index and edge features
    edge_index = []
    edge_features = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
        edge_features.append([
            bond.GetBondTypeAsDouble(),
            int(bond.GetIsConjugated()),
            int(bond.IsInRing())
        ])
        edge_features.append([
            bond.GetBondTypeAsDouble(),
            int(bond.GetIsConjugated()),
            int(bond.IsInRing())
        ])

    if len(edge_index) == 0:
        print(f"SMILES has no bonds: {smiles}")
        return None

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_features = torch.tensor(edge_features, dtype=torch.float)

    # Create a PyTorch Geometric Data object
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    return data

# Apply function to convert SMILES to graph and filter out invalid ones
#train_df = pd.DataFrame({'Structure (SMILES)': ['CCO', 'O=C=O', 'CCC']})  # Dummy DataFrame
train_df['Graph'] = train_df['Structure (SMILES)'].apply(smiles_to_graph)
data_list = [graph for graph in train_df['Graph'] if graph is not None]  # Filter out None values



class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, edge_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.edge_mlp = nn.Linear(edge_dim, hidden_dim)
        self.pool = global_mean_pool
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Initialize with a small positive bias
        self.conv1.bias = nn.Parameter(torch.ones(hidden_dim) * 0.01)
        self.conv2.bias = nn.Parameter(torch.ones(hidden_dim) * 0.01)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Normalize input features
        x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-5)

        edge_features = self.edge_mlp(edge_attr)
        x = F.leaky_relu(self.conv1(x, edge_index))
        x = F.leaky_relu(self.conv2(x, edge_index))
        x = self.pool(x, data.batch)
        return self.fc(x).squeeze()

def train_gnn(model, data_list, targets, epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data, target in zip(data_list, targets):
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(data_list)}')

# Recreate the model and train
gnn_model = GNN(input_dim=6, hidden_dim=32, output_dim=1, edge_dim=3)
targets = torch.randn(len(data_list))  # Replace with actual targets
train_gnn(gnn_model, data_list, targets, epochs=100, lr=0.001)

# Generate embeddings
def generate_embeddings(model, data_list):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for data in data_list:
            x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
            x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-5)
            x = F.leaky_relu(model.conv1(x, edge_index))
            x = F.leaky_relu(model.conv2(x, edge_index))
            embedding = model.pool(x, data.batch)
            embeddings.append(embedding.cpu().numpy())
    return embeddings

embeddings = generate_embeddings(gnn_model, data_list)

# Print statistics about the embeddings
import numpy as np
embeddings_array = np.array(embeddings)
print(f"Embeddings shape: {embeddings_array.shape}")
print(f"Mean: {np.mean(embeddings_array)}")
print(f"Std: {np.std(embeddings_array)}")
print(f"Min: {np.min(embeddings_array)}")
print(f"Max: {np.max(embeddings_array)}")
'''

In [ ]:
### Graph features
def one_hot_encoding(x, permitted_list):
    if x not in permitted_list:
        x = permitted_list[-1]
    return [int(x == s) for s in permitted_list]

def get_atom_features(atom, use_chirality=True, hydrogens_implicit=True):
    permitted_list_of_atoms = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'Unknown']
    if not hydrogens_implicit:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms

    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    is_in_a_ring_enc = [int(atom.IsInRing())]
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    atomic_mass_scaled = [(atom.GetMass() - 10.812) / 116.092]
    vdw_radius_scaled = [(Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5) / 0.6]
    covalent_radius_scaled = [(Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64) / 0.76]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + \
                          is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + \
                          covalent_radius_scaled

    if use_chirality:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc

    if hydrogens_implicit:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

def get_bond_features(bond, use_stereochemistry=True):
    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc

    if use_stereochemistry:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES: {smiles}")
        return None

    node_features = []
    for atom in mol.GetAtoms():
        node_features.append(get_atom_features(atom))
    node_features = torch.tensor(node_features, dtype=torch.float)

    edge_indices = []
    edge_features = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices.extend([[i, j], [j, i]])
        edge_features.extend([get_bond_features(bond)] * 2)

    if len(edge_indices) == 0:
        print(f"SMILES has no bonds: {smiles}")
        return None

    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_features = torch.tensor(edge_features, dtype=torch.float)

    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    return data


In [ ]:
#GCN
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, edge_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.edge_mlp = nn.Linear(edge_dim, hidden_dim)
        self.pool = global_mean_pool

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-5)
        edge_features = self.edge_mlp(edge_attr)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.pool(x, data.batch)

def generate_embeddings(model, data_list):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for data in data_list:
            embedding = model(data)
            embeddings.append(embedding.cpu().numpy())
    return embeddings

NameError: name 'nn' is not defined

In [ ]:
# Embeddings for training data

train_df['Graph'] = train_df['Structure (SMILES)'].apply(smiles_to_graph)
data_list = [graph for graph in train_df['Graph'] if graph is not None]

# Initialize the model
input_dim = data_list[0].x.shape[1]
edge_dim = data_list[0].edge_attr.shape[1]
gcn_model = GCN(input_dim=input_dim, hidden_dim=32, edge_dim=edge_dim)

# Generate embeddings
embeddings = generate_embeddings(gcn_model, data_list)

# Print statistics about the embeddings
embeddings_array = np.array(embeddings)
print(f"Embeddings shape: {embeddings_array.shape}")
print(f"Mean: {np.mean(embeddings_array)}")
print(f"Std: {np.std(embeddings_array)}")
print(f"Min: {np.min(embeddings_array)}")
print(f"Max: {np.max(embeddings_array)}")

# Save embeddings
#np.save('molecular_embeddings.npy', embeddings_array)

Embeddings shape: (697, 1, 32)
Mean: 0.07732225209474564
Std: 0.07413024455308914
Min: 0.0
Max: 0.4506397247314453


In [ ]:
y = train_df['Class']
y

,Class
0,1
1,1
2,1
3,1
4,1
...,...
692,0
693,0
694,0
695,0


In [ ]:
import torch.optim as optim  # Add this import for the optimizer

# Assuming the necessary imports and functions are defined earlier (like smiles_to_graph)

# Convert SMILES to graph data
train_df['Graph'] = train_df['Structure (SMILES)'].apply(smiles_to_graph)

# Filter out invalid graphs (if any)
data_list = [graph for graph in train_df['Graph'] if graph is not None]

# Initialize the GCN model
input_dim = data_list[0].x.shape[1]  # Input feature dimension
edge_dim = data_list[0].edge_attr.shape[1]  # Edge feature dimension
gcn_model = GCN(input_dim=input_dim, hidden_dim=32, edge_dim=edge_dim)

# Optimizer
optimizer = optim.Adam(gcn_model.parameters(), lr=0.001)

# Loss function (assuming classification task)
criterion = torch.nn.CrossEntropyLoss()

# DataLoader to batch the graphs
train_loader = DataLoader(data_list, batch_size=32, shuffle=True)
y = train_df['Class']
# Training loop
gcn_model.train()  # Set the model in training mode (enables dropout if used)
for epoch in range(100):
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()  # Reset gradients from the previous step
        out = gcn_model(data)  # Forward pass through the model
        loss = criterion(out, data.y)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters




/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not NoneType

In [ ]:
# Assuming data_list is already created and contains valid graph data

# Check if data_list is not empty
if data_list:
    # Get the first graph from the list
    first_graph = data_list[2]

    # Extract node features (atom features)
    node_features = first_graph.x

    # Extract edge features (bond features)
    edge_features = first_graph.edge_attr

    print("Node (Atom) Features:")
    print(f"Shape: {node_features.shape}")
    print("First node feature vector:")
    print(node_features[0])

    print("\nEdge (Bond) Features:")
    print(f"Shape: {edge_features.shape}")
    print("First edge feature vector:")
    print(edge_features[0])

    # Optionally, you can print the meanings of each feature
    atom_feature_names = [
        "Atom type (one-hot encoded)",
        "Number of heavy neighbors (one-hot encoded)",
        "Formal charge (one-hot encoded)",
        "Hybridization type (one-hot encoded)",
        "Is in ring",
        "Is aromatic",
        "Atomic mass (scaled)",
        "Van der Waals radius (scaled)",
        "Covalent radius (scaled)",
        "Chirality (one-hot encoded)",
        "Number of hydrogens (one-hot encoded)"
    ]

    bond_feature_names = [
        "Bond type (one-hot encoded)",
        "Is conjugated",
        "Is in ring",
        "Stereochemistry (one-hot encoded)"
    ]

    print("\nAtom Feature Meanings:")
    for i, name in enumerate(atom_feature_names):
        print(f"{i}: {name}")

    print("\nBond Feature Meanings:")
    for i, name in enumerate(bond_feature_names):
        print(f"{i}: {name}")
else:
    print("data_list is empty. Please ensure that valid graph data has been generated.")

Node (Atom) Features:
Shape: torch.Size([22, 79])
First node feature vector:
tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0103, 0.3333, 0.1579, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000])

Edge (Bond) Features:
Shape: torch.Size([48, 10])
First edge feature vector:
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

Atom Feature Meanings:
0: Atom type (one-hot encoded)
1: Number of heavy neighbors (one-ho

In [ ]:
cols = [f'emb_{i}' for i in range(embeddings_array.shape[2])]
embeddings_array_2d = embeddings_array.reshape(embeddings_array.shape[0], embeddings_array.shape[2])
embeddings_array_df = pd.DataFrame(embeddings_array_2d, columns=cols)
#print(embeddings_array_df.head())

train_df_cp = train_df.copy()
train_df_cp = train_df_cp.drop('Structure (SMILES)', axis=1)
train_df_cp = train_df_cp.drop('Graph', axis=1)
train_df_cp = train_df_cp.drop('Drug Name', axis=1)

def preprocess_classification(X):
    X.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    scaler = MinMaxScaler()
    #scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    return X_scaled

y = train_df_cp['Class']
y_df = pd.DataFrame(y, columns=['Class'])  # Name your target column as 'Target_Class' (or any name you prefer)

original_features = preprocess_classification(train_df_cp.iloc[:, 0:5])
original_features_df = pd.DataFrame(original_features, columns = ['Drug pKa',	'Log P',	'Half Life (hrs-1)',	'No of Rotatable Bonds',	'M.Wt'])
fea_emb = pd.concat([original_features_df, embeddings_array_df], axis=1)
train_fea_emb_df = pd.concat([fea_emb, y_df], axis =1 ) # original_features + embeddings

features_mol = preprocess_classification(train_df_cp.iloc[:,0:-1])
features_mol_df = pd.DataFrame(features_mol, columns = train_df_cp.columns[0:-1])
fea_emb_df = pd.concat([features_mol_df, embeddings_array_df], axis=1)
gnn_mol_features_train_df = pd.concat([fea_emb_df, y_df], axis =1 ) #features + molecular descriptors + embeddings

atc_gnn_mol_features_train = pd.concat([fea_emb_df, train_atc_norm_df], axis =1 )
atc_gnn_mol_features_train_df = pd.concat([atc_gnn_mol_features_train, y_df], axis =1 )  #features + molecular descriptors + embeddings +atc

In [ ]:
# Assuming you've already trained your model on the training set

# Load your test data
test_df['Graph'] = test_df['Structure (SMILES)'].apply(smiles_to_graph)
test_data_list = [graph for graph in test_df['Graph'] if graph is not None]

# Initialize the model
input_dim = test_data_list[0].x.shape[1]
edge_dim = test_data_list[0].edge_attr.shape[1]
gnn_model = GCN(input_dim=input_dim, hidden_dim=32, edge_dim=edge_dim)

# Generate embeddings
test_embeddings = generate_embeddings(gnn_model, test_data_list)

# Print statistics about the embeddings
test_embeddings_array = np.array(test_embeddings)
print(f"Embeddings shape: {test_embeddings_array.shape}")
print(f"Mean: {np.mean(test_embeddings_array)}")
print(f"Std: {np.std(test_embeddings_array)}")
print(f"Min: {np.min(test_embeddings_array)}")
print(f"Max: {np.max(test_embeddings_array)}")

# Save embeddings
#np.save('molecular_embeddings.npy', embeddings_array)


Embeddings shape: (45, 1, 32)
Mean: 0.08334869891405106
Std: 0.07598452270030975
Min: 0.0
Max: 0.45660170912742615


In [ ]:
cols = [f'emb_{i}' for i in range(test_embeddings_array.shape[2])]
embeddings_array_2d = test_embeddings_array.reshape(test_embeddings_array.shape[0], test_embeddings_array.shape[2])
embeddings_array_df = pd.DataFrame(embeddings_array_2d, columns=cols)
#print(embeddings_array_df.head())


def preprocess_classification(X):
    X.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    scaler = MinMaxScaler()
    #scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    return X_scaled

test_df_cp = test_df.copy()
test_df_cp = test_df_cp.drop('Structure (SMILES)', axis=1)
test_df_cp = test_df_cp.drop('Graph', axis=1)
test_df_cp = test_df_cp.drop('Drug Name', axis=1)

y = test_df_cp['Class']
y_df = pd.DataFrame(y, columns=['Class'])  # Name your target column as 'Target_Class' (or any name you prefer)

original_features = preprocess_classification(test_df_cp.iloc[:, 0:5])
original_features_df = pd.DataFrame(original_features, columns = ['Drug pKa',	'Log P',	'Half Life (hrs-1)',	'No of Rotatable Bonds',	'M.Wt'])
fea_emb = pd.concat([original_features_df, embeddings_array_df], axis=1)
test_fea_emb_df = pd.concat([fea_emb, y_df], axis =1 ) # original_features + embeddings

test_features_mol = preprocess_classification(test_df_cp.iloc[:,0:-1])
test_features_mol_df = pd.DataFrame(test_features_mol, columns = test_df_cp.columns[0:-1])

test_combined_df = pd.concat([test_features_mol_df, embeddings_array_df], axis=1)
gnn_mol_features_test_df = pd.concat([test_combined_df, y_df], axis =1 ) # original_features + embeddings + descriptors

atc_gnn_mol_features_test = pd.concat([test_combined_df, test_atc_norm_df], axis = 1)
atc_gnn_mol_features_test_df = pd.concat([atc_gnn_mol_features_test, y_df], axis =1 ) #original_features + embeddings + descriptors + atc

In [ ]:
##Graph Sage

from torch_geometric.nn import SAGEConv, global_mean_pool
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdchem
import torch.nn as nn

def one_hot_encoding(x, permitted_list):
    if x not in permitted_list:
        x = permitted_list[-1]
    return [int(x == s) for s in permitted_list]

def get_atom_features(atom, use_chirality=True, hydrogens_implicit=True):
    permitted_list_of_atoms = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'Unknown']
    if not hydrogens_implicit:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms

    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    is_in_a_ring_enc = [int(atom.IsInRing())]
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    atomic_mass_scaled = [(atom.GetMass() - 10.812) / 116.092]
    vdw_radius_scaled = [(Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5) / 0.6]
    covalent_radius_scaled = [(Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64) / 0.76]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + \
                          is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + \
                          covalent_radius_scaled

    if use_chirality:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc

    if hydrogens_implicit:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

def get_bond_features(bond, use_stereochemistry=True):
    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc

    if use_stereochemistry:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES: {smiles}")
        return None

    node_features = []
    for atom in mol.GetAtoms():
        node_features.append(get_atom_features(atom))
    node_features = torch.tensor(node_features, dtype=torch.float)

    edge_indices = []
    edge_features = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices.extend([[i, j], [j, i]])
        edge_features.extend([get_bond_features(bond)] * 2)

    if len(edge_indices) == 0:
        print(f"SMILES has no bonds: {smiles}")
        return None

    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_features = torch.tensor(edge_features, dtype=torch.float)

    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    return data


class GraphSAGE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, edge_dim):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.edge_mlp = nn.Linear(edge_dim, hidden_dim)
        self.pool = global_mean_pool
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # Normalize input features
        x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-5)

        # Apply edge features
        edge_features = self.edge_mlp(edge_attr)

        # First GraphSAGE layer
        x = F.relu(self.conv1(x, edge_index))

        # Second GraphSAGE layer
        x = F.relu(self.conv2(x, edge_index))

        # Global pooling
        x = self.pool(x, batch)

        # Final linear layer
        return self.fc(x)

def train_graphsage(model, data_list, targets, epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data, target in zip(data_list, targets):
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out.squeeze(), target.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(data_list)}')

def generate_embeddings(model, data_list):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for data in data_list:
            x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
            x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-5)
            x = F.relu(model.conv1(x, edge_index))
            x = F.relu(model.conv2(x, edge_index))
            embedding = model.pool(x, batch)
            embeddings.append(embedding.cpu().numpy())
    return embeddings

# Example usage
train_df['Graph'] = train_df['Structure (SMILES)'].apply(smiles_to_graph)
data_list = [graph for graph in train_df['Graph'] if graph is not None]
targets = torch.tensor(train_df['Class'].values)

input_dim = data_list[0].x.shape[1]
edge_dim = data_list[0].edge_attr.shape[1]
graphsage_model = GraphSAGE(input_dim=input_dim, hidden_dim=64, output_dim=1, edge_dim=edge_dim)

# Train the model
train_graphsage(graphsage_model, data_list, targets, epochs=100, lr=0.001)

# Generate embeddings for training set
embeddings = generate_embeddings(graphsage_model, data_list)

# Print statistics about the embeddings
embeddings_array = np.array(embeddings)
print(f"Embeddings shape: {embeddings_array.shape}")
print(f"Mean: {np.mean(embeddings_array)}")
print(f"Std: {np.std(embeddings_array)}")
print(f"Min: {np.min(embeddings_array)}")
print(f"Max: {np.max(embeddings_array)}")



In [ ]:
# parameter settings code
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score

# Dense model
class DenseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_rate=0.5):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class PyTorchModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=64, lr=0.01, epochs=100, dropout_rate=0.5):
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.lr = lr
        self.epochs = epochs
        self.dropout_rate = dropout_rate  # Add dropout_rate
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.classes_ = np.array([0, 1])  # Binary classification

    def _initialize_model(self):
        return DenseModel(self.input_dim, self.hidden_dim1, self.hidden_dim2, 1, self.dropout_rate).to(self.device)

    def fit(self, X, y):
        self.model = self._initialize_model()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.BCEWithLogitsLoss()
        dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).view(-1, 1))
        loader = DataLoader(dataset, batch_size=32, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        return self

    def predict_proba(self, X):
        if self.model is None:
            raise ValueError("Model has not been fitted yet.")
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            outputs = self.model(X_tensor)
            probs = torch.sigmoid(outputs).cpu().numpy()
        return np.hstack([1 - probs, probs])  # Return probabilities for both classes

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

def custom_cross_val_score(estimator, X, y, cv, scoring):
    scores = []
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        y_pred_proba = estimator.predict_proba(X_test)[:, 1]

        if scoring == 'accuracy':
            score = accuracy_score(y_test, y_pred)
        elif scoring == 'roc_auc':
            score = roc_auc_score(y_test, y_pred_proba)
        elif scoring == 'balanced_accuracy':
            score = balanced_accuracy_score(y_test, y_pred)
        elif scoring == 'recall':
            score = recall_score(y_test, y_pred)
        elif scoring == 'precision':
            score = precision_score(y_test, y_pred)
        elif scoring == 'f1':
            score = f1_score(y_test, y_pred)

        scores.append(score)

    return np.array(scores)

def run_grid_search(X_train, y_train):
    param_grid = {
        'hidden_dim1': [32, 64, 128, 256],
        'hidden_dim2': [16, 32, 64],
        'lr': [0.001, 0.0001, 0.01],
        'dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5]
    }

    clf = PyTorchModelWrapper(input_dim=X_train.shape[1])
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Write all results to a text file
    with open("/content/drive/MyDrive/HOB/Rerun/grid_search_results_gcn_fm.txt", "w") as f:
        f.write("Results for PyTorch classifier:\n\n")
        for mean_score, params in zip(grid_search.cv_results_["mean_test_score"], grid_search.cv_results_["params"]):
            f.write(f"Mean accuracy: {mean_score:.4f}, Params: {params}\n")

    best_params = grid_search.best_params_
    final_clf = clf.set_params(**best_params)

    results = {}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for metric in ['accuracy', 'roc_auc', 'balanced_accuracy', 'recall', 'precision', 'f1']:
        scores = custom_cross_val_score(final_clf, X_train, y_train, cv, scoring=metric)
        results[metric.capitalize()] = scores.mean()

    # Kappa score
    kappa_scores = []
    for train_index, test_index in cv.split(X_train, y_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
        final_clf.fit(X_train_fold, y_train_fold)
        y_pred = final_clf.predict(X_test_fold)
        kappa = cohen_kappa_score(y_test_fold, y_pred)
        kappa_scores.append(kappa)
    results["Kappa"] = np.mean(kappa_scores)

    # Append best results to the text file
    with open("/content/drive/MyDrive/HOB/Rerun/grid_search_results_gcn_fm.txt", "a") as f:
        f.write("\nBest Parameters:\n")
        for key, value in best_params.items():
            f.write(f"{key}: {value}\n")
        f.write("\nBest Results:\n")
        for key, value in results.items():
            f.write(f"{key}: {value:.4f}\n")

    # Save best results to Excel file
    df = pd.DataFrame([best_params | results])
    df.to_excel("/content/drive/MyDrive/HOB/Rerun/best_results_gcn_fm.xlsx", index=False)

    return best_params, results

def main():
    # Load your data
    X_train_tensor = torch.tensor(gnn_mol_features_train_df.values[:, :-1], dtype=torch.float32)
    y_train_tensor = torch.tensor(gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)

    # Convert to numpy for GridSearchCV
    X_train = X_train_tensor.numpy()
    y_train = y_train_tensor.numpy().ravel()

    # Run grid search
    best_params, results = run_grid_search(X_train, y_train)

    print("\nBest parameters and results have been saved to 'grid_search_results_gcn_fm.txt' and 'best_results_gcn_fm.xlsx'")

if __name__ == "__main__":
    main()



Best parameters and results have been saved to 'grid_search_results_gcn_fm.txt' and 'best_results_gcn_fm.xlsx'


In [ ]:
'''
X_train_tensor = torch.tensor(train_fea_emb_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(train_fea_emb_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(test_fea_emb_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(test_fea_emb_df['Class'].values, dtype=torch.float32).view(-1, 1)'''

X_train_tensor = torch.tensor(atc_gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(atc_gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(atc_gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(atc_gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)
'''
X_train_tensor = torch.tensor(gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)'''

"\nX_train_tensor = torch.tensor(gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)\ny_train_tensor = torch.tensor(gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)\nX_test_tensor = torch.tensor(gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)\ny_test_tensor = torch.tensor(gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)"

In [ ]:
#Cross validation
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt

class DenseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_rate=0.5):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_dense_model(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds).flatten(), np.array(all_labels).flatten()

def cross_validate_dense_model(X, y, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, n_splits=5, n_repeats=10, epochs=100, batch_size=32):
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)
    metrics = {
        'accuracy': [], 'balanced_accuracy': [], 'precision': [],
        'recall': [], 'f1': [], 'auc': [], 'kappa' : []
    }
    all_roc_data = []

    dataset = TensorDataset(torch.FloatTensor(X), torch.FloatTensor(y))

    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        print(f"Fold {fold + 1}/{n_splits * n_repeats}")

        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)

        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

        model = DenseModel(input_dim, hidden_dim1, hidden_dim2, output_dim)
        optimizer = optim.Adam(model.parameters(), lr= 0.0001)

        train_dense_model(model, train_loader, criterion, optimizer, epochs)

        val_preds, val_labels = evaluate_model(model, val_loader)

        metrics['accuracy'].append(accuracy_score(val_labels, val_preds > 0.5))
        metrics['balanced_accuracy'].append(balanced_accuracy_score(val_labels, val_preds > 0.5))
        metrics['precision'].append(precision_score(val_labels, val_preds > 0.5))
        metrics['recall'].append(recall_score(val_labels, val_preds > 0.5))
        metrics['f1'].append(f1_score(val_labels, val_preds > 0.5))
        metrics['auc'].append(roc_auc_score(val_labels, val_preds))
        metrics['kappa'].append(cohen_kappa_score(val_labels, val_preds > 0.5))

        fpr, tpr, _ = roc_curve(val_labels, val_preds)
        all_roc_data.append((fpr, tpr, metrics['auc'][-1]))

        print(f"Fold {fold + 1} - Accuracy: {metrics['accuracy'][-1]:.4f}, Balanced Accuracy: {metrics['balanced_accuracy'][-1]:.4f}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    std_metrics = {metric: np.std(scores) for metric, scores in metrics.items()}

    print("\nAverage Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f} ± {std_metrics[metric]:.4f}")

    # Plot ROC curves
    plt.figure(figsize=(10, 10))
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.zeros_like(mean_fpr)
    for roc in all_roc_data:
        fpr, tpr, auc_score = roc
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr /= len(all_roc_data)
    mean_auc = np.mean([roc[2] for roc in all_roc_data])
    plt.plot(mean_fpr, mean_tpr, lw=2, label=f'Mean ROC (AUC = {mean_auc:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='r', lw=2, label='Random Guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=24)
    plt.ylabel('True Positive Rate', fontsize=24)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=24)
    plt.legend(loc="lower right", fontsize=18)
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/HOB/Rerun/roc_curve_mol.png', format='png', bbox_inches='tight')
    plt.savefig('/content/drive/MyDrive/HOB/Rerun/roc_curve_mol.eps', format='eps', bbox_inches='tight')
    plt.close()

    return avg_metrics, std_metrics

# Usage
input_dim = X_train_tensor.shape[1]
hidden_dim1 = 32
hidden_dim2 = 16
output_dim = 1
criterion = nn.BCEWithLogitsLoss()

avg_metrics, std_metrics = cross_validate_dense_model(
    X_train_tensor.numpy(), y_train_tensor.numpy(),
    input_dim, hidden_dim1, hidden_dim2, output_dim, criterion,
    n_splits=5, n_repeats=10, epochs=100, batch_size=32
)
def save_metrics_to_excel(avg_metrics, std_metrics, filename='metrics.xlsx'):
    # Combine avg and std metrics into a single DataFrame
    metrics_df = pd.DataFrame({
        'Metric': list(avg_metrics.keys()),
        'Average': list(avg_metrics.values()),
        'Standard Deviation': list(std_metrics.values())
    })

    # Save the DataFrame to an Excel file
    metrics_df.to_excel(filename, index=False)

save_metrics_to_excel(avg_metrics, std_metrics, '/content/drive/MyDrive/HOB/Rerun/metrics.xlsx')

Fold 1/50
Epoch 10/100, Loss: 0.5931341217623817
Epoch 20/100, Loss: 0.5369629876481162
Epoch 30/100, Loss: 0.4966844502422545
Epoch 40/100, Loss: 0.46197225981288487
Epoch 50/100, Loss: 0.4216574645704693
Epoch 60/100, Loss: 0.3713489721218745
Epoch 70/100, Loss: 0.316034912235207
Epoch 80/100, Loss: 0.26691240982876885
Epoch 90/100, Loss: 0.24263381212949753
Epoch 100/100, Loss: 0.2061309094230334
Fold 1 - Accuracy: 0.8000, Balanced Accuracy: 0.6061
Fold 2/50
Epoch 10/100, Loss: 0.6421741876337264
Epoch 20/100, Loss: 0.5548690325684018
Epoch 30/100, Loss: 0.5140771485037274
Epoch 40/100, Loss: 0.49797484775384265
Epoch 50/100, Loss: 0.4615841077433692
Epoch 60/100, Loss: 0.42403654754161835
Epoch 70/100, Loss: 0.38398225605487823
Epoch 80/100, Loss: 0.3348555548323525
Epoch 90/100, Loss: 0.301211675008138
Epoch 100/100, Loss: 0.26275601651933456
Fold 2 - Accuracy: 0.8000, Balanced Accuracy: 0.6407
Fold 3/50
Epoch 10/100, Loss: 0.6170327597194247
Epoch 20/100, Loss: 0.5525340735912323

In [ ]:
#Different sampling techniques using cross validation
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler


class DenseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_rate=0.5):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_dense_model(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds).flatten(), np.array(all_labels).flatten()


def cross_validate_dense_model(X, y, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion,
                               n_splits=5, n_repeats=10, epochs=100, batch_size=32, sampling_method='smote'):
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    metrics = {
        'accuracy': [], 'balanced_accuracy': [], 'precision': [],
        'recall': [], 'f1': [], 'auc': [], 'kappa': []
    }
    all_roc_data = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        print(f"Fold {fold + 1}/{n_splits * n_repeats}")

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Apply sampling method
        if sampling_method == 'ros':
            sampler = RandomOverSampler(random_state=42)
        elif sampling_method == 'rus':
            sampler = RandomUnderSampler(random_state=42)
        elif sampling_method == 'smote':
            sampler = SMOTE(random_state=42)
        else:
            raise ValueError("Invalid sampling method. Choose 'ros', 'rus', or 'smote'.")

        X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

        # Convert to PyTorch tensors
        train_dataset = TensorDataset(torch.FloatTensor(X_train_resampled), torch.FloatTensor(y_train_resampled))
        val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        model = DenseModel(input_dim, hidden_dim1, hidden_dim2, output_dim)
        optimizer = optim.Adam(model.parameters(), lr= 0.0001)

        train_dense_model(model, train_loader, criterion, optimizer, epochs)

        val_preds, val_labels = evaluate_model(model, val_loader)

        metrics['accuracy'].append(accuracy_score(val_labels, val_preds > 0.5))
        metrics['balanced_accuracy'].append(balanced_accuracy_score(val_labels, val_preds > 0.5))
        metrics['precision'].append(precision_score(val_labels, val_preds > 0.5))
        metrics['recall'].append(recall_score(val_labels, val_preds > 0.5))
        metrics['f1'].append(f1_score(val_labels, val_preds > 0.5))
        metrics['auc'].append(roc_auc_score(val_labels, val_preds))
        metrics['kappa'].append(cohen_kappa_score(val_labels, val_preds > 0.5))

        fpr, tpr, _ = roc_curve(val_labels, val_preds)
        all_roc_data.append((fpr, tpr, metrics['auc'][-1]))

        print(f"Fold {fold + 1} - Accuracy: {metrics['accuracy'][-1]:.4f}, Balanced Accuracy: {metrics['balanced_accuracy'][-1]:.4f}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    std_metrics = {metric: np.std(scores) for metric, scores in metrics.items()}

    print("\nAverage Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f} ± {std_metrics[metric]:.4f}")

    # Plot ROC curves
    plt.figure(figsize=(10, 10))
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.zeros_like(mean_fpr)
    for roc in all_roc_data:
        fpr, tpr, auc_score = roc
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr /= len(all_roc_data)
    mean_auc = np.mean([roc[2] for roc in all_roc_data])
    plt.plot(mean_fpr, mean_tpr, lw=2, label=f'Mean ROC (AUC = {mean_auc:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='r', lw=2, label='Random Guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=24)
    plt.ylabel('True Positive Rate', fontsize=24)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=24)
    plt.legend(loc="lower right", fontsize=18)
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/HOB/Rerun/roc_curve_mol.png', format='png', bbox_inches='tight')
    plt.savefig('/content/drive/MyDrive/HOB/Rerun/roc_curve_mol.eps', format='eps', bbox_inches='tight')
    plt.close()

    return avg_metrics, std_metrics

# Usage
input_dim = X_train_tensor.shape[1]
hidden_dim1 = 32
hidden_dim2 = 16
output_dim = 1
criterion = nn.BCEWithLogitsLoss()

avg_metrics, std_metrics = cross_validate_dense_model(
    X_train_tensor.numpy(), y_train_tensor.numpy(),
    input_dim, hidden_dim1, hidden_dim2, output_dim, criterion,
    n_splits=5, n_repeats=10, epochs=100, batch_size=32, sampling_method='smote'
)

save_metrics_to_excel(avg_metrics, std_metrics, '/content/drive/MyDrive/HOB/Rerun/metrics.xlsx')

Fold 1/50
Epoch 10/100, Loss: 0.6807075628527889
Epoch 20/100, Loss: 0.6289697532300595
Epoch 30/100, Loss: 0.526373831210313
Epoch 40/100, Loss: 0.4035666408362212
Epoch 50/100, Loss: 0.32668699931215356
Epoch 60/100, Loss: 0.2544406216453623
Epoch 70/100, Loss: 0.21775850846811576
Epoch 80/100, Loss: 0.1632593439684974
Epoch 90/100, Loss: 0.1386447482638889
Epoch 100/100, Loss: 0.11701670509797556
Fold 1 - Accuracy: 0.7429, Balanced Accuracy: 0.5842
Fold 2/50
Epoch 10/100, Loss: 0.6807695222752435
Epoch 20/100, Loss: 0.6186227394001824
Epoch 30/100, Loss: 0.5046769987259593
Epoch 40/100, Loss: 0.37596595713070463
Epoch 50/100, Loss: 0.3027234109384673
Epoch 60/100, Loss: 0.22623135814709322
Epoch 70/100, Loss: 0.18413542636803218
Epoch 80/100, Loss: 0.1463113416518484
Epoch 90/100, Loss: 0.12367848839078631
Epoch 100/100, Loss: 0.09959333642785038
Fold 2 - Accuracy: 0.7429, Balanced Accuracy: 0.6824
Fold 3/50
Epoch 10/100, Loss: 0.6679619462401779
Epoch 20/100, Loss: 0.59071739956184

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, roc_curve, classification_report
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Usage
'''
X_train_tensor = torch.tensor(atc_gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(atc_gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(atc_gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(atc_gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)'''

X_train_tensor = torch.tensor(gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)


class DenseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_rate=0.3):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_dense_model(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds).flatten(), np.array(all_labels).flatten()

def run_experiment(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, epochs=100, batch_size=32):
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = DenseModel(input_dim, hidden_dim1, hidden_dim2, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    train_dense_model(model, train_loader, criterion, optimizer, epochs)

    test_preds, test_labels = evaluate_model(model, test_loader)

    metrics = {
        'accuracy': accuracy_score(test_labels, test_preds > 0.5),
        'balanced_accuracy': balanced_accuracy_score(test_labels, test_preds > 0.5),
        'precision': precision_score(test_labels, test_preds > 0.5),
        'recall': recall_score(test_labels, test_preds > 0.5),
        'f1': f1_score(test_labels, test_preds > 0.5),
        'auc': roc_auc_score(test_labels, test_preds),
        'kappa': cohen_kappa_score(test_labels, test_preds > 0.5)
    }

    class_report = classification_report(test_labels, test_preds > 0.5, output_dict=True)

    return metrics, class_report, model, test_preds, test_labels

def main(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, n_runs=10, epochs=100, batch_size=32):
    all_metrics = []
    best_accuracy = 0
    best_model = None
    best_class_report = None
    best_predictions = None
    best_labels = None
    best_metrics = None

    for run in range(n_runs):
        print(f"Run {run + 1}/{n_runs}")
        metrics, class_report, model, test_preds, test_labels = run_experiment(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, epochs, batch_size)
        all_metrics.append(metrics)

        if metrics['accuracy'] > best_accuracy:
            best_accuracy = metrics['accuracy']
            best_model = model
            best_class_report = class_report
            best_predictions = test_preds
            best_labels = test_labels
            best_metrics = metrics

    # Calculate average metrics
    avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()
    std_metrics = pd.DataFrame(all_metrics).std().to_dict()

    # Create dataframes for average and best results
    avg_df = pd.DataFrame([avg_metrics])
    best_df = pd.DataFrame([best_metrics])

    # Ensure the output directory exists
    os.makedirs('/content/drive/MyDrive/HOB/Rerun', exist_ok=True)

    # Save average results to CSV
  #  avg_df.to_csv('/content/drive/MyDrive/HOB/average_results.csv', index=False)

    # Save best results to CSV
 #   best_df.to_csv('/content/drive/MyDrive/HOB/best_results.csv', index=False)

    # Save best predictions in a text file
 #   np.savetxt('/content/drive/MyDrive/HOB/best_predictions.txt', best_predictions, fmt='%.4f')

    # Save best class-level results
#    with open('/content/drive/MyDrive/HOB/best_class_results.txt', 'w') as f:
  #      f.write(str(best_class_report))

    # Print average results
    print("\nAverage Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f} ± {std_metrics[metric]:.4f}")

    return avg_metrics, std_metrics, best_model, avg_df, best_df, best_predictions, best_labels


input_dim = X_train_tensor.shape[1]
hidden_dim1 = 64
hidden_dim2 = 64
output_dim = 1
criterion = nn.BCEWithLogitsLoss()

avg_metrics, std_metrics, best_model, avg_df, best_df, best_predictions, best_labels = main(
    X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    input_dim, hidden_dim1, hidden_dim2, output_dim, criterion,
    n_runs=10, epochs=100, batch_size=32
)

# Save the best model
torch.save(best_model.state_dict(), '/content/drive/MyDrive/HOB/Rerun/best_model.pth')

# Print dataframes
print("\nAverage Results Dataframe:")
print(avg_df)
print("\nBest Results Dataframe:")
print(best_df)

# Create a dataframe with best predictions and true labels
best_predictions_df = pd.DataFrame({
    'True_Label': best_labels,
    'Predicted_Probability': best_predictions,
    'Predicted_Class': (best_predictions > 0.5).astype(int)
})
best_predictions_df.to_csv('/content/drive/MyDrive/HOB/Rerun/class_fm_run10.csv', index=False)
avg_df.to_csv('/content/drive/MyDrive/HOB/Rerun/avg.csv', index=False)
best_df.to_csv('/content/drive/MyDrive/HOB/Rerun/best.csv', index=False)


Run 1/10
Epoch 10/100, Loss: 0.4739733257076957
Epoch 20/100, Loss: 0.4651192453774539
Epoch 30/100, Loss: 0.4574689946391366
Epoch 40/100, Loss: 0.43904648450287903
Epoch 50/100, Loss: 0.44070514088327234
Epoch 60/100, Loss: 0.4320393909107555
Epoch 70/100, Loss: 0.4420002644712275
Epoch 80/100, Loss: 0.4211132973432541
Epoch 90/100, Loss: 0.41651548919352616
Epoch 100/100, Loss: 0.4239954244006764
Run 2/10
Epoch 10/100, Loss: 0.4761085103858601
Epoch 20/100, Loss: 0.4551822150295431
Epoch 30/100, Loss: 0.481801145456054
Epoch 40/100, Loss: 0.44245425137606537
Epoch 50/100, Loss: 0.4314325736327605
Epoch 60/100, Loss: 0.41641678593375464
Epoch 70/100, Loss: 0.44077564775943756
Epoch 80/100, Loss: 0.40816370194608514
Epoch 90/100, Loss: 0.40381878208030353
Epoch 100/100, Loss: 0.4141152873635292
Run 3/10
Epoch 10/100, Loss: 0.484448949044401
Epoch 20/100, Loss: 0.45472208342768927
Epoch 30/100, Loss: 0.46384744210676715
Epoch 40/100, Loss: 0.4300312873992053
Epoch 50/100, Loss: 0.44595

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, roc_curve, classification_report
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Usage
'''
X_train_tensor = torch.tensor(atc_gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(atc_gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(atc_gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(atc_gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)'''

X_train_tensor = torch.tensor(gnn_mol_features_train_df.values[:,:-1], dtype=torch.float32)
y_train_tensor = torch.tensor(gnn_mol_features_train_df['Class'].values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(gnn_mol_features_test_df.values[:,:-1], dtype=torch.float32)
y_test_tensor = torch.tensor(gnn_mol_features_test_df['Class'].values, dtype=torch.float32).view(-1, 1)


class DenseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_rate=0.3):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_dense_model(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds).flatten(), np.array(all_labels).flatten()

def run_experiment(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, epochs=100, batch_size=32, sampling_method='smote'):
    # Convert tensors to numpy arrays for resampling
    X_train_np = X_train.cpu().numpy()
    y_train_np = y_train.cpu().numpy().ravel()
    X_test_np = X_test.cpu().numpy()
    y_test_np = y_test.cpu().numpy().ravel()

    # Apply sampling method
    if sampling_method == 'ros':
        sampler = RandomOverSampler(random_state=42)
    elif sampling_method == 'rus':
        sampler = RandomUnderSampler(random_state=42)
    elif sampling_method == 'smote':
        sampler = SMOTE(random_state=42)
    else:
        raise ValueError("Invalid sampling method. Choose 'ros', 'rus', or 'smote'.")

    X_train_resampled, y_train_resampled = sampler.fit_resample(X_train_np, y_train_np)
    X_test_resampled, y_test_resampled = X_test_np, y_test_np  # We don't resample the test set

    # Convert back to tensors
    X_train_tensor = torch.tensor(X_train_resampled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_resampled, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test_resampled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_resampled, dtype=torch.float32).view(-1, 1)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = DenseModel(input_dim, hidden_dim1, hidden_dim2, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    train_dense_model(model, train_loader, criterion, optimizer, epochs)

    test_preds, test_labels = evaluate_model(model, test_loader)

    metrics = {
        'accuracy': accuracy_score(test_labels, test_preds > 0.5),
        'balanced_accuracy': balanced_accuracy_score(test_labels, test_preds > 0.5),
        'precision': precision_score(test_labels, test_preds > 0.5),
        'recall': recall_score(test_labels, test_preds > 0.5),
        'f1': f1_score(test_labels, test_preds > 0.5),
        'auc': roc_auc_score(test_labels, test_preds),
        'kappa': cohen_kappa_score(test_labels, test_preds > 0.5)
    }

    class_report = classification_report(test_labels, test_preds > 0.5, output_dict=True)

    return metrics, class_report, model, test_preds, test_labels

def main(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, n_runs=10, epochs=100, batch_size=32):
    all_metrics = []
    best_accuracy = 0
    best_model = None
    best_class_report = None
    best_predictions = None
    best_labels = None
    best_metrics = None

    for run in range(n_runs):
        print(f"Run {run + 1}/{n_runs}")
        metrics, class_report, model, test_preds, test_labels = run_experiment(X_train, y_train, X_test, y_test, input_dim, hidden_dim1, hidden_dim2, output_dim, criterion, epochs, batch_size, sampling_method = 'smote')
        all_metrics.append(metrics)

        if metrics['accuracy'] > best_accuracy:
            best_accuracy = metrics['accuracy']
            best_model = model
            best_class_report = class_report
            best_predictions = test_preds
            best_labels = test_labels
            best_metrics = metrics

    # Calculate average metrics
    avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()
    std_metrics = pd.DataFrame(all_metrics).std().to_dict()

    # Create dataframes for average and best results
    avg_df = pd.DataFrame([avg_metrics])
    best_df = pd.DataFrame([best_metrics])

    # Ensure the output directory exists
    os.makedirs('/content/drive/MyDrive/HOB/', exist_ok=True)

    # Save average results to CSV
  #  avg_df.to_csv('/content/drive/MyDrive/HOB/average_results.csv', index=False)

    # Save best results to CSV
 #   best_df.to_csv('/content/drive/MyDrive/HOB/best_results.csv', index=False)

    # Save best predictions in a text file
 #   np.savetxt('/content/drive/MyDrive/HOB/best_predictions.txt', best_predictions, fmt='%.4f')

    # Save best class-level results
#    with open('/content/drive/MyDrive/HOB/best_class_results.txt', 'w') as f:
  #      f.write(str(best_class_report))

    # Print average results
    print("\nAverage Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f} ± {std_metrics[metric]:.4f}")

    return avg_metrics, std_metrics, best_model, avg_df, best_df, best_predictions, best_labels


input_dim = X_train_tensor.shape[1]
hidden_dim1 = 64
hidden_dim2 = 64
output_dim = 1
criterion = nn.BCEWithLogitsLoss()

avg_metrics, std_metrics, best_model, avg_df, best_df, best_predictions, best_labels = main(
    X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    input_dim, hidden_dim1, hidden_dim2, output_dim, criterion,
    n_runs=10, epochs=100, batch_size=32
)

# Save the best model
torch.save(best_model.state_dict(), '/content/drive/MyDrive/HOB/Rerun/best_model.pth')

# Print dataframes
print("\nAverage Results Dataframe:")
print(avg_df)
print("\nBest Results Dataframe:")
print(best_df)

# Create a dataframe with best predictions and true labels
best_predictions_df = pd.DataFrame({
    'True_Label': best_labels,
    'Predicted_Probability': best_predictions,
    'Predicted_Class': (best_predictions > 0.5).astype(int)
})
best_predictions_df.to_csv('/content/drive/MyDrive/HOB/Rerun/class_smotefm_run10.csv', index=False)
avg_df.to_csv('/content/drive/MyDrive/HOB/Rerun/avg.csv', index=False)
best_df.to_csv('/content/drive/MyDrive/HOB/Rerun/best.csv', index=False)


Run 1/10
Epoch 10/100, Loss: 0.6074029005625668
Epoch 20/100, Loss: 0.5756185019717497
Epoch 30/100, Loss: 0.5547844294239493
Epoch 40/100, Loss: 0.5379039855564341
Epoch 50/100, Loss: 0.5122784604044521
Epoch 60/100, Loss: 0.4980377248104881
Epoch 70/100, Loss: 0.49251872870851965
Epoch 80/100, Loss: 0.4860232595135184
Epoch 90/100, Loss: 0.48765866633723765
Epoch 100/100, Loss: 0.4553864747285843
Run 2/10
Epoch 10/100, Loss: 0.604026652434293
Epoch 20/100, Loss: 0.5698247490560308
Epoch 30/100, Loss: 0.5398666420403648
Epoch 40/100, Loss: 0.5438099372036317
Epoch 50/100, Loss: 0.5224456418963039
Epoch 60/100, Loss: 0.5149677886682398
Epoch 70/100, Loss: 0.5233369715073529
Epoch 80/100, Loss: 0.4939783522311379
Epoch 90/100, Loss: 0.47895097469582276
Epoch 100/100, Loss: 0.5076786577701569
Run 3/10
Epoch 10/100, Loss: 0.6030277434517356
Epoch 20/100, Loss: 0.5726405504871818
Epoch 30/100, Loss: 0.5621038219508003
Epoch 40/100, Loss: 0.5355889446595136
Epoch 50/100, Loss: 0.52497243442